In [1]:
import base64
import json
import requests

Get Token

In [2]:
token_get_input = {
    "user_api_email": "",
    "username_api": "", # same as the email provided
    "user_api_password": "",
    "token": ""
}

In [ ]:
request_url = "" # GET TOKEN URL
exchange0_output = requests.post(request_url, json=token_get_input)
print(exchange0_output.content)

In [ ]:
access_token = json.loads(exchange0_output.content.decode("utf-8"))["access_token"]
print(access_token)

Summarize PDF

In [5]:
exchange1_input = {
    "input_type": "pdf", # optional, default is pdf
    "streaming_url_as_data": True, # optional, default is False
    "data": "", # required string
    "section_entities": True, # all 5 categories of output are optional and default False
    "section_summaries": True, # so you could skip this key, say
    "meta_summary": True,
    "meta_entities": True,
    "patient_metadata": True,
    "optimize_sections": True,
    "scoring": True, # optional, default is False
    "token": access_token,
    "user_api_email": ""
    # other params listed in the API documentation will roll out slowly, and will all default to current behavior to be backward compatible
}

In [6]:
with open("your_pdf.pdf", "rb") as f:
    pdf = f.read()
    # it does NOT matter what the current encoding of the pdf is, always base64 then utf-8
    exchange1_input["data"] = base64.b64encode(pdf).decode("utf-8")
    exchange1_input["streaming_url_as_data"] = False

In [ ]:
request_url = "" # SUMMARY REQUEST URL
exchange1_output = requests.post(request_url, json=exchange1_input)
print(exchange1_output.content)

conversation_id = exchange1_output.json()["conversation_id"]

In [ ]:
print(exchange1_output)

Alternatively you may pass us a presigned url, or a public url in the "data" field.


Then pass it as a plain string, and make sure streaming_url_as_input is "True".


There's no payload limit either way. The 2 options are purely for your convenience. 


Each PDF should be 200 pages or less.

In [16]:
exchange2_input = {
    "conversation_id": conversation_id, # the conversation_id you got from exchange1_output,
    "user_api_email": "",
    "token": access_token, 
}

In [ ]:
retrieve_url = "" # SUMMARY RETRIEVE URL
exchange2_output = requests.post(retrieve_url, json=exchange2_input)
print(exchange2_output.content)
# this may be done or not done
# please don't check too frequently, as checking has no bearing on speeding up the process.
# checking frequency: recommend every minute, but will probably be fine every 15 seconds.

In [ ]:
# suppose it is done, you get a presigned url.
output_json = requests.get(exchange2_output.json()["url"]).content
print(output_json)

In [ ]:
with open("output.json", "wb") as f:
    f.write(output_json)

Summarize CDA

In [5]:
exchange1_input = {
    "input_type": "cda", # optional, default is pdf
    "streaming_url_as_data": True, # optional, default is False
    "data": "", # required string
    "section_entities": True, # all 5 categories of output are optional and default False
    "section_summaries": True, # so you could skip this key, say
    "meta_summary": True,
    "meta_entities": True,
    "patient_metadata": True,
    "optimize_sections": False, # optional, default is False
    "token": access_token,
    "scoring": True,
    "user_api_email": ""
    # other params listed in the API documentation will roll out slowly, and will all default to current behavior to be backward compatible
}

In [6]:
with open("your_cdas.txt", "rb") as f:
    # txt should be cdas concatenated, <ClinicalDocument> objects one by one
    # we use regex to parse out the ClinicalDocument objects, so whitespace or headers between objects are fine
    cda = f.read()
    # base64 then utf-8
    exchange1_input["data"] = base64.b64encode(cda).decode("utf-8")
    exchange1_input["streaming_url_as_data"] = False

In [ ]:
request_url = "" # SUMMARY REQUEST URL
exchange1_output = requests.post(request_url, json=exchange1_input)
print(exchange1_output.content)

conversation_id = exchange1_output.json()["conversation_id"]

In [8]:
exchange2_input = {
    "conversation_id": conversation_id, # the conversation_id you got from exchange1_output,
    "user_api_email": "",
    "token": access_token, 
}

In [ ]:
retrieve_url = "" # SUMMARY RETRIEVE URL
exchange2_output = requests.post(retrieve_url, json=exchange2_input)
print(exchange2_output.content)
# retrieval policy is the same as pdf

In [ ]:
# suppose it is done, you get a presigned url.
output_json = requests.get(exchange2_output.json()["url"]).content
print(output_json)

Summarize FHIR

In [ ]:
exchange1_input = {
    "input_type": "fhir", # optional, default is pdf
    "streaming_url_as_data": True, # optional, default is False
    "data": "", # required string
    "section_entities": True, # all 5 categories of output are optional and default False
    "section_summaries": True, # so you could skip this key, say
    "meta_summary": True,
    "meta_entities": True,
    "patient_metadata": True,
    "optimize_sections": False, # optional, default is False
    "token": access_token,
    "user_api_email": ""
    # other params listed in the API documentation will roll out slowly, and will all default to current behavior to be backward compatible
}
with open("your_json.json", "rb") as f:
    # a json list of fhir bundles
    '''
    [
        {
            "fhirResource": {
                "resourceType": "Bundle",
                "id": "",
                "meta": {
                    "lastUpdated": ""
                },
                "entry": []
            }
        },
        {
            "fhirResource": {
                "resourceType": "Bundle",
                "id": "",
                "meta": {
                    "lastUpdated": ""
                },
                "entry": []
            }
        }
    ]
    '''
    fhir = f.read()
    # base64 then utf-8
    exchange1_input["data"] = base64.b64encode(fhir).decode("utf-8")
    exchange1_input["streaming_url_as_data"] = False
request_url = "" # SUMMARY REQUEST URL
exchange1_output = requests.post(request_url, json=exchange1_input)
print(exchange1_output.content)

conversation_id = exchange1_output.json()["conversation_id"]
print(conversation_id)

In [ ]:
exchange2_input = {
    "conversation_id": conversation_id, # the conversation_id you got from exchange1_output,
    "user_api_email": "",
    "token": access_token, 
}
retrieve_url = "" # SUMMARY RETRIEVE URL
exchange2_output = requests.post(retrieve_url, json=exchange2_input)
print(exchange2_output.content)
# suppose it is done, you get a presigned url.
output_json = requests.get(exchange2_output.json()["url"]).content
print(output_json)